In [1]:
import numpy as np
from matplotlib import pyplot as plt

import rawpy, PIL
import pickle
from glob import glob

rawpy_params = {'use_camera_wb':True, 'no_auto_scale':False, 
                'no_auto_bright':True, 'chromatic_aberration':(1,1)}

# Concepts behind calibration frames

When capturing the night sky, the objects we try to capture are often so faint that photon counts end up being dominated not by the source we want to image, but rather photons from everything else and even imperfections in our own camera. If we were to process raw photographs of the sky we would find numerous issues from grainy images, to bad colors, to vignette. All of these will detract from the final nebula or galaxy.

Thankfully, science has our back, and we've developed rather simplistic techniques with incredible potency in increasing the fidelity of our final image which end up being the exact same techniques used to create images from Hubble or even JWST!  To start, we need to understand the theory behind what our camera is actually doing when taking an image.

## What is an image?

While older cameras might use photographic film which crystalizes when exposed to light, modern cameras leverage Charge-Coupled Devices (CCD). These tiny circuits, when exposed to light will generate a small charge proportional to the amount of light hitting the surface. As more light hits it, more charge builds up. Then, when the camera closes, the charge is held in place until a 'shift register' mechanism moves all of the charge to a device which can measure the amount of charge that each pixel recieved. This means that the amount of charge read is roughly proportional to the amount of light that pixel recived and therefor, the brightness of that pixel in the resulting image, $b$, is proportional to the amount of light recieved. 

$$ b \propto q \propto n_{\rm{photons}} $$

In a perfect world, this would be the only equation you would need for astrophotography. Sadly, imperfection is the name of the game and we have to deal with noise too. To quantify a particular pixel's response to all sources of noise, we need an equation which ecapsulates them all. We can do that through the following equation:

$$ \sigma^2_{\rm{total}}(t) = \sigma^2_{\rm{thermal}}(T)(t) + \sigma^2_{\rm{read}} + \sigma^2_{\rm{sky}}(t) + \sigma^2_{\rm{source}}(t) $$


There are 4 sources of noise in this equation:

1. Thermal Noise
2. Read noise
3. Background (sky) noise
4. Source (shot) noise 


### Thermal noise

When taking an image, a camera measures the built up charge of a CCD. While ideally this charge would result purely from photons from our source, we can get all sorts of other processes that can create a residual charge as well. Cheif among these is thermal processes. Since the detector itself has some non-zero temperature, thanks to blackbody radiation, the CCD and other parts of your camera will produce some amount of photon radiation. When these photons land on the CCD, just as if they were a galaxy, the CCD will build up charge. In general the higher the temperature of the camera, the more thermal noise you will find.

In astrophotography, we can figure out the average amount of thermal noise is in our camera through calibration frames often called "Darks" or "Dark frames". These must be taken at the same temperature that the Lights or Light frames were taken and with the exact same settings including focal length/zoom, focus, exposure time, ISO and white balance, though this time, the goal isn't to image anything at all. Instead they should be taken with the lens cap **on**. It may sound weird, but if we want to figure out how much thermal noise we expect in our CCD, we need to count how much thermal noise we have in the absense of any typical sources of light.

###


## Constructing a master bias frame

In order to calculate how much noise our CCD creates when reading in the data from each pixel, we need to load in all of our bias frames and combine them.

blah, blah


Medians should be more robust to hot pixels, but we can try both.
- Maybe there's a way to calculate the median of the pixels without loading all of them in, but I don't know it and couldn't find it online
- Averages can absolutely be calculated without loading them in, but if we want to clip extrema, we need to anyway

In [3]:
# Glob all the bias frames
bias_files = glob('../photos/bias/*.CR3')
nbiases = len(bias_files)

all_biases = None
for i in range(nbiases): 
    with rawpy.imread(bias_files[i]) as raw:
        rgb = raw.postprocess(**rawpy_params)

    if all_biases is None:
        shape = (nbiases, rgb.shape[0], rgb.shape[1], 3)
        all_biases = np.zeros(shape, dtype=np.uint8) # Save memory. Just use uint8
    
    all_biases[i] = rgb

In [4]:
all_biases[0].dtype

dtype('uint8')

In [6]:
np.median(all_biases, axis=0).astype(np.float16)

array([[[0. , 0. , 0. ],
        [0.5, 0. , 0. ],
        [0. , 0. , 0. ],
        ...,
        [0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ]],

       [[0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ],
        ...,
        [0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ]],

       [[0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ],
        ...,
        [0. , 0.5, 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0.5]],

       ...,

       [[0. , 0. , 0. ],
        [0.5, 0. , 0. ],
        [1. , 0. , 0. ],
        ...,
        [0.5, 0. , 0. ],
        [0.5, 0. , 0. ],
        [0.5, 0. , 0. ]],

       [[0. , 0. , 0.5],
        [0. , 0. , 1. ],
        [0.5, 0. , 0. ],
        ...,
        [1. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ]],

       [[0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [1. , 0. , 0. ],
        ...,
        [0.5, 0. , 0. ],
        [1. , 0. , 0. ],
        [1. , 0. , 0. ]]

In [2]:
# Medians

bias_median = np.median(all_biases, axis=0)

plt.imshow(bias_median)
plt.title('Bias Median')
plt.show()


# Averages with sigma clipping
#sigma_clip = 3
#std = np.std(all_biases, axis=0)

#clipped = all_biases.copy()
#clipped[np.abs(all_biases - np.average(all_biases, axis=0)) > sigma_clip*std] = np.nan


#bias_mean = np.average(clipped, axis=0, nan_policy='omit')


NameError: name 'all_biases' is not defined